In [1]:
import numpy as np

In [98]:
class DCM(object):
    def __init__(self, 
                 dcm_id="dcm00001",
                 a = None,
                 b = None,
                 c = None,
                 ons = None, # list containing vector of onsets (in scans) for every input
                 dur = None, # list containing vector of durations (in scans) for every input
                 names = None, # names of brain regions
                 n=None, # number of regions in model
                 v=None, # number of scans                 
                 m=None, # number of inputs (experimental conditions)
                 x = None, # number of states per region
                 **kwargs):
        self.dcm_id = dcm_id
        self.a = a
        self.b = b
        self.c = c
        self.ons = ons
        self.dur = dur
        self.n = n
        self.names = names        
        self.v = v
        self.m = m
        self.input_names = input_names  
        self.x = x
        self._set_items(**kwargs)
        
    def _set_items(self, **kwargs):
        for k, v in list(kwargs.items()):
            setattr(self, k, v)
    def __getitem__(self, key):
        return self.__dict__[key]
    def __repr__(self):
        return repr(self.__dict__)
    
    
def dcmParam(a=None, # vector of length n*n representing the anatomical connections between
#                         brain regions. The 'a' vector should look like an n*n matrix with a non zero
#                         element (in Hz.) on location ij when there is an expected connection from
#                         region j (column) to region i (row)
             b=None, # vector of length n*n*m representing the functional connections. Should
#                         be written as m n*n matrices whith non zero elements ijk (in Hz.) if input k
#                         influences the connection from region j (column) to region i (row) and zero
#                         otherwise
             c=None, # vector of length n*m representing the input connections. Should be written as 
#                         an m by n matrix whith non zero elements kj (in Hz.) if input k
#                         (row) influences region j (columns)
             n=None, # number of regions in model                         
             m=None, # number of inputs (experimental conditions)
             names=list(), # names of brain regions
             input_names=list(), # names of inputs
             driven_input_names = list(),
             modulatory_input_names = list(),
             auto = True # if True the DCM object (a,b,c matrices)  need not be constructed 
#                          in a step by step procedure from params else DCM (a,b,c matrices) is specified 
#                          and only need to be written as class instance and to construct the HPF and SF
#              ------------------------------------------------------------------------------------------
#              v=None, # number of scans 
#              ons = list(), # list containing vector of onsets (in scans) for every input
#              dur = list(), # list containing vector of durations (in scans) for every input             
#              TR = 0.72, # repetition time of the experiment (default = 0.72 s)
#              TE = 0.033, #  echo time in seconds (default = 0.033 s)
#              T = 16, # number of timebins (default = 16)
#               h = c(0.65, 0.41, 0.98, 0.32, 0.34, 0), # the parameters of the hemodynamic model
#              HPF=0, # High pass filter in seconds (default = 0 seconds)
#              x = 2 * n, # number of states (= 2 times number of regions) 
             
            ):
#     @TODO allow to specify all params not only full model
    
    if (auto == True):
        
        dcm = {}
        
        if not n:
            warnings.warn(
                "No number of regions given, param n should be specified") 
            return  
        dcm['n'] = n
        
        if len(names) == 0:
            dcm['names']  = ['reg%s'%i for i in range(n)]
        else:
            dcm['names'] = names
        
        if not m:
            warnings.warn(
                "No number of inputs is given, param m should be specified, DCM cannot be specified without inputs") 
            return  
        else:
            dcm['m'] = m 
            
        if len(input_names) == 0:
            dcm['input_names']  = ['inp%s'%i for i in range(m)]
        else:
            dcm['input_names'] = input_names

#         dcm['TR'] = TR
#         dcm['TE'] = TE
        
#         if len(ons) == 0:
#             warnings.warn(
#                 "No onset is given, param ons should be specified") 
#             return  

#         if len(dur) == 0:
#             warnings.warn(
#                 "No duration is given, param dur should be specified") 
#             return  
       
#         if m!=len(ons):
#             warnings.warn(
#                 "Given onsets doesn't correspond to inputs number. Should be specified for each input") 
#             return 
#         else:
#             dcm['ons'] = ons  
            
#         if m!=len(dur):
#             warnings.warn(
#                 "Given duration doesn't correspond to inputs number. Should be specified for each input") 
#             return  
#         else:
#             dcm['dur'] = dur          

#         if not x:
#             dcm['x'] = 1
#         else:
#             dcm['x'] = x
            
#         if not v:
#             warnings.warn(
#                 "No number of scans is given, param v should be specified") 
#             return  
#         else:
#             dcm['v'] = v  
        

#         default: full connectivity
#         A = np.ones((n,n)) # full connectivity
#         default: neighbours are connected
        A = np.eye(n) + np.diag(np.ones(n-1), 1) + np.diag(np.ones(n-1), -1) 
        dcm['a'] = A
           
#         default: if modulatory_input_names is empty all inputs influence on all extrinsic connections
#                     else if modulatory_input_names is given only inputs in list influence on all extrinsic connections
        B = [[]]*m
        for i,name in zip(range(m),input_names):
            if len(modulatory_input_names) == 0:
                B[i] = A + (np.eye(n)*-1)
            else:
                if name in modulatory_input_names:
                    B[i] = A + (np.eye(n)*-1)                    
                else:
                    B[i] = np.zeros((n, n))
        dcm['b']  = B

#         default: if driven_input_names is empty all inputs influence on all regions
#                     else if driven_input_names is given only inputs in list influence on all regions
        
        if len(driven_input_names) == 0:
            C = np.ones((n,m)) # full connectivity
        else:
            C = []
            for i,name in zip(range(m),input_names):
                if name in driven_input_names:
                    c = np.ones(n)
                else:
                    c = np.zeros(n)
                C.append(c)
        C = np.transpose(C)
        dcm['c']  = C
        
        dcm_instance = DCM(dcm_id="dcm00001",
                             a = dcm['a'],
                             b = dcm['b'],
                             c = dcm['c'],
        #                      ons = None, # list containing vector of onsets (in scans) for every input
        #                      dur = None, # list containing vector of durations (in scans) for every input
                             names = names, # names of brain regions
                             n=dcm['n'], # number of regions in model
        #                      v=None, # number of scans                 
                             m=dcm['m'], # number of inputs (experimental conditions)
        #                      x = None
                        )
        
        return dcm_instance

# @TODO не заводить словарь внутри

In [99]:
dcm = dcmParam(n=2, m = 3, names=['r1','r2'], input_names=['i1','i2'],
         modulatory_input_names=['i1'], driven_input_names=['i2'])


In [100]:
dcm

{'x': None, 'c': array([[ 0.,  1.],
       [ 0.,  1.]]), 'm': 3, 'names': ['r1', 'r2'], 'b': [array([[ 0.,  1.],
       [ 1.,  0.]]), array([[ 0.,  0.],
       [ 0.,  0.]]), []], 'input_names': ['i1', 'i2'], 'dcm_id': 'dcm00001', 'v': None, 'dur': None, 'n': 2, 'ons': None, 'a': array([[ 1.,  1.],
       [ 1.,  1.]])}

In [103]:
dcm['a']

array([[ 1.,  1.],
       [ 1.,  1.]])

In [101]:
dcm['b']

[array([[ 0.,  1.],
        [ 1.,  0.]]), array([[ 0.,  0.],
        [ 0.,  0.]]), []]

In [102]:
dcm['c']

array([[ 0.,  1.],
       [ 0.,  1.]])

In [106]:
dcm.a != 0

array([[ True,  True],
       [ True,  True]], dtype=bool)

In [ ]:
def spm_dcm_priors(DCM):
'''
INPUT:
    a,b,c,d - constraints on connections (1 - present, 0 - absent)

OUTPUT:
    pE     - prior expectations (connections and hemodynamic)
    pC     - prior covariances  (connections and hemodynamic)
    x      - prior (initial) states
'''    
    n= DCM.n
    
    pA = 64
    dA = 1
    
#     prior expectations
    if isvector(A):
        A     = logical(A);
        pE.A  = (A(:) - 1)*dA;
    else
        A     = logical(A - diag(diag(A)));
        pE.A  = A/128;
    
    
    a0 = 1 if DCM.a!=0 else 0
    b0 = 1 if DCM.b!=0 else 0
    c0 = 1 if DCM.c!=0 else 0
    
    p = 1e-3
    b = 1
    s = 3.0902 # fixed, but never used?
    q = qchisq((1-p),DCM.n*(DCM.n-1))
    q = DCM.n/((DCM.n - 1)*q)
    
    #Intrinsic connections
    a0 = matrix(a0,ncol=DCM.n, byrow=TRUE)  
    diag(a0) = 0  
    dim(b0) = c(DCM.n,DCM.n,DCM.m)
    for i in range(DCM.m)
        b0[:,:,i] = t(b0[:,:,i])*q
              
    pC = diag(c(1/16, a0*q, b0, c0)) # Covariance neuronal state
    
    #Expectations
    Ae = diag(-1,DCM.n)
    Be = 0 #ifelse (DCM.b!=0,0,0)
    Ce = 0 #ifelse (DCM.c!=0,0,0)
    pE = cbind(c(log(b),Ae,Be,Ce)) # Expectation neuron state
    
    qE = DCM.h # Expectation hemod. state
    
    qC= matrix(c(1.4837866e-2,-1.2823236e-4,-4.0541540e-4,3.4889257e-4,-3.4493093e-4,0,
    -1.2823236e-4,1.8370175e-3,-7.9879877e-4,4.2061309e-4,-1.1577819e-5,0,
    -4.0541540e-4,-7.9879877e-4,5.1387416e-2,2.0607856e-3,1.7781387e-3,0,
    3.4889257e-4,4.2061309e-4,2.0607856e-3,2.0564973e-4,6.6429753e-5,0,
    -3.4493093e-4,-1.1577819e-5,1.7781387e-3,6.6429753e-5,6.9002931e-5,0,
    0,0,0,0,0,3.1250000e-2),ncol=6)
    
    qC = qC%x%diag(1,DCM.n)
    qE = qE%x%cbind(rep(1,DCM.n))
    pE = rbind(pE,qE)
       
    pC = as.matrix(bdiag(pC,qC))
    p = list()
    p.pC=pC
    p.pE=pE
    p.qC=qC
    p.qE=qE
    return p
    


In [115]:
np.log(2)

0.69314718055994529

In [116]:
def dcmCompare(DCM1, DCM2):
    bf = {}
#     'AIC overall Bayes Factor'
    nats =  -1*(DCM1.AIC-DCM2.AIC)
    bits = nats/np.log(2)
    bf_aic = 2**(-bits)
    bf.bf_aic = bf_aic

#     'BIC overall Bayes Factor'
    nats =  -1*(DCM1.BIC-DCM2.BIC)
    bits = nats/log(2)
    bf_bic = 2**(-bits)
    bf.bf_bic = bf_bic
    
    return bf
    

In [ ]:
def dcmEvidence(DCM,ts):
    #DCM.X0 = 1
    DCM.priors=spm_dcm_priors(DCM)
    if (len(DCM.X0)==0){
        X0 = matrix(rep(1,DCM.v))}
    else
        X0=as.matrix(DCM.X0)
    y = spm_int(PM=DCM.Ep,DCM=DCM)
    R = ts-y
    R = R - X0%*%solve(t(X0)%*%X0)%*%(t(X0)%*%R)
    pCdiag = diag(DCM.priors.pC)
    wsel = which(pCdiag!=0)
    evidence_region_cost = numeric(DCM.n)
    for i in range(DCM.n):
        lambda =  as.matrix(DCM.Ce)[i*DCM.v,i*DCM.v]
        evidence_region_cost[i] = -.5*DCM.v*log(lambda)
        evidence_region_cost[i] =  evidence_region_cost[i] - ((.5*t(R[,i])*(1/lambda))%*%diag(DCM.v))%*%R[,i]
    
    evidence_aic_penalty = length(wsel)
    evidence_bic_penalty = .5*length(wsel)*log(DCM.v)
    
    DCM.AIC = sum(evidence_region_cost)- evidence_aic_penalty
    DCM.BIC = sum(evidence_region_cost)- evidence_bic_penalty
    
    return DCM
    

In [2]:
def stimfun(DCM):
    TR = DCM.TR
    T= DCM.T
    v = DCM.v
    sf = list()
    for i in range(1,len(DCM.ons) + 1):
    
        if (len(DCM.dur[[i]])==1):
             DCM.dur[[i]] = rep(DCM.dur[[i]], len(DCM.ons[[i]]))
    
        dur = DCM.dur[[i]]
        ons = DCM.ons[[i]]

        if dur==0:
            u_sf = T/TR
        else:
            u_sf = 1
    
        ton = round(ons*T)+32
        tof = round(dur*T) + ton + 1
    
        sf[[i]] = rep(0,DCM.v*T+128)
        ton = pmax(ton,1)
        tof = pmax(tof,1)
        for j in range(1,len(ton)+1):
            if (len(sf[[i]]) > ton[j]):
                sf[[i]][ton[j]] = sf[[i]][ton[j]] + u_sf
            if (len(sf[[i]]) > tof[j]):
                sf[[i]][tof[j]] = sf[[i]][tof[j]] - u_sf
        sf[[i]] = cumsum(sf[[i]])
        sf[[i]] = sf[[i]][1:(DCM.v*T + 32)]
        
    
    stimfunc  = matrix(NA,len(sf[[1]]), length(DCM.ons))
    for i in range(1,len(DCM.ons)):
        stimfunc[,i] = sf[[i]]
    
    #prepare sf for DCM (drop first 32 timepoints)
    
    sf = as.matrix(stimfunc[33:nrow(stimfunc),])
    s = ceiling((1:v)*nrow(sf)/v) # output times
    if (ncol(sf)>1):    
        t = NA
        for i in range(1,ncol(sf)+1):
            temp = c(1, which(diff(sf[,i])!=0)+1)
            t = c(t,temp) # input times
                               
        t = sort(t[-1])[-which(diff(sort(t[-1]))==0)]
    
    if (ncol(sf)==1):
        t=c(1,which(diff(sf)!=0)+1) # input times
        
    T0 = sort(c(s,t))
    s = sort(c(s,t),index.return=TRUE).ix
    dt0 = c(TR/T*diff(T0),0)

    DCM.sf = sf
    DCM.s = s
    DCM.T0 = T0
    DCM.dt0 = dt0
    return DCM

In [ ]:
def dcmEstimate(DCM=DCM,ts):
    
    DCM = stimfun(DCM)
    DCM.priors = spm_dcm_priors(DCM) 
    pC = DCM.priors.pC
    pE = DCM.priors.pE 
    
    if (len(DCM.X0)==0):
#     X0 = HPF(DCM)#   
#     x0 = matrix(rep(1,DCM.v))
#     if (length(X0)>1){ 
#     X0 = as.matrix(cbind(x0,X0))
#     } 
        X0 = matrix(rep(1,DCM.v))    
    else:
        X0 = as.matrix(DCM.X0)
    
    
    ## spm_nlsi
    nr = length(ts)/DCM.v
    
    nh = DCM.n
    nt = len(ts)
    nq = nr*DCM.v/nt
    h = rep(0,nh)
    nb = dim(X0)[1]
    nx = nr*DCM.v/nb
    dfdu = diag(nx)%x%X0
    Q = diag(nx)%x%rep(1,DCM.v)
    hE = rep(0,nh)
    ihC = diag(nh)
    
    ##   ## svd priors.pC in R or ...##
    V = spm_svd(pC,exp(-16))
    V2 = Matrix(V)
    
    ## ... Copy decomposed matrix from Matlab
    
    ## spm_nlsi ##
    nu = dim(dfdu)[2]
    np = dim(V)[2]
    ip = 1:np
    iu = (1:nu)+np
    
    ## 2nd order moments
    pC    = t(V2)%*%pC%*%V2
    uC    = diag(nu)/1e-8
    ipC = solve(bdiag(pC,uC))
    # initialize conditional density
    Eu    = solve(t(dfdu)%*%dfdu)%*%(t(dfdu)%*%c(ts))
    p     = rBind(t(V)%*%(pE-pE), Eu)
    Ep    = pE + V%*%p[ip]
    
    Cp    = pC
    
    ## EM
    CF   = -Inf
    t    = 256
     dFdh = rep(0,nh)
     dFdhh = matrix(0,nh,nh)
     Pr = vector('list',length=nh)
     PS = vector('list',length=nh)
    dx = exp(-8)
    dfdp=matrix(0,DCM.v*nr,np)
 options(warn=-1)
 for(ka in 1:64){# 1 ipv 128 om te testen


    ####  spm_int ##########
    
    f0 = spm_int(PM=Ep,DCM=DCM)
    
    
	
	for (i in 1:dim(dfdp)[2]){
	xmi = Ep+V2[,i]*dx
	fi = spm_int(PM=xmi,DCM)
	dfdp[,i]  = spm_dfdx(fi,f0,dx)
	}
    
    
    # prediction error and full gradients
    e     = Matrix(c(ts) - c(f0) - dfdu%*%p[iu])
      
#	J     = cBind(-dfdp, -dfdu)
    J     = Matrix(cBind(-dfdp, -dfdu))
    
    
    ## M-STEP
    
	for (im in 1:16){
	#precision and conditional covariance

        iS = Diagonal(nt)*1e-8
	    for (i in 1:nh){
	      iS = iS + Diagonal(x=Q[,i])*exp(h[i])
	      }
	  S     = solve(iS)
		iS    = diag(nq)%x%iS
		Cp    = solve(t(J)%*%iS%*%J + ipC) 
	        
      
   for (i in 1:nh){
	    Pr[[i]] = Diagonal(x=Q[,i])*exp(h[i])
	    PS[[i]] = Pr[[i]]%*%S
	    Pr[[i]] = diag(nq)%x%Pr[[i]]
	    }
	
	# derivatives: dLdh = dL/dh,...
	   
    for (i in 1:nh){
	    dFdh[i] = as.numeric(sum(diag(PS[[i]]))*nq/2-as.numeric(t(e)%*%Pr[[i]])%*%e/2-sum(Cp*(t(J)%*%Pr[[i]])%*%J)/2)
	    
		for (j in i:nh){
		dFdhh[i,j] = -sum(colSums(as.matrix(PS[[i]]*PS[[j]])))*nq/2
		dFdhh[j,i] =  dFdhh[i,j]
		}
	    }

    

	# add hyperpriors
	d     = h - hE;
	dFdh  = dFdh  - ihC%*%d;
	dFdhh = dFdhh - ihC;  
	#update ReML estimate
	Ch    = solve(-dFdhh)
	dh    = Ch%*%dFdh
	h     = h  + dh
	    for(i in 1:nh){
	    h[i]     = max(h[i],-16)
	    }
	dF    = t(dFdh)%*%dh
	if( as.numeric(dF) < 10^-2){break}
	}
    
    ## E-STEP
    
	
    F = as.numeric(-1*t(e)%*%iS%*%e/2- crossprod(p,ipC)%*%p/2 - crossprod(d,ihC)%*%d/2 - DCM.v*nr*log(8*atan(1))/2 - logdet(S)*nq/2 + logdet(ipC%*%Cp)/2 + logdet(ihC%*%Ch)/2)
	
	if(F>CF){
	C_p   = p
	C_h   = h
	CF   = F
	      # E-Step: Conditional update of gradients and curvature
	dFdp  = t(-J)%*%iS%*%e - ipC%*%p
	dFdpp = t(-J)%*%iS%*%J - ipC
	# decrease regularization
	t     = t*2
	str = paste("EM-step(-):")
	}else{
	p     = C_p
	h     = C_h
	    # and increase regularization
	t     = min(t/2,128)
	str = paste("EM-step(+):")
	}
	
    ## E-STEP update
    dp    = spm_dx(dFdpp,dFdp,t)
    
    p  = p + dp
    Ep = pE + V%*%p[ip]

    ## Convergence
    dF = as.numeric(crossprod(dFdp,dp))
    cat(str, ka,'   F:',CF,'  dF:',dF,"\n")
    if (ka > 2 && dF < 1e-2){break}
    }

    
    DCM.Ep = Ep
	Ep = Ep[-1]
	j = 1:(DCM.n*DCM.n)
    DCM.A = matrix(Ep[j],DCM.n,DCM.n)
	if(length(DCM.names)==DCM.n) {colnames(DCM.A)=DCM.names
	rownames(DCM.A)=DCM.names}
	
	Ep = Ep[-j]
    j = 1:(DCM.n*DCM.n*DCM.m)
    B0 = Ep[j]
    dim(B0) = c(DCM.n,DCM.n,DCM.m)
    DCM.B = B0
if(length(DCM.names)==DCM.n){
for (z in 1:dim(DCM.B)[3]){
colnames(DCM.B[,,z])=DCM.names
	rownames(DCM.B[,,z])=DCM.names}
}
    Ep = Ep[-j]
    j = 1:(DCM.n*DCM.m)
    DCM.C = t(matrix(Ep[j],DCM.n,DCM.m))
    if(length(DCM.names)==DCM.n) {
    colnames(DCM.C)=DCM.names
    rownames(DCM.C) = names(DCM.ons)}
    Ep = Ep[-j]
    DCM.H = matrix(Ep,nrow=DCM.n)   
DCM.Cp = as.matrix(V%*%Cp[ip,ip]%*%t(V))
    PP = Ncdf(DCM.Ep,diag(DCM.Cp))
 
	PP = PP[-1]
	j = 1:(DCM.n*DCM.n)
    DCM.pA = matrix(PP[j],DCM.n,DCM.n)
	if(length(DCM.names)==DCM.n) {colnames(DCM.pA)=DCM.names
	rownames(DCM.pA)=DCM.names}
	
	PP = PP[-j]
    j = 1:(DCM.n*DCM.n*DCM.m)
    BB = PP[j]
    dim(BB) = c(DCM.n,DCM.n,DCM.m)
    DCM.pB = BB
if(length(DCM.names)==DCM.n){
for (z in 1:dim(DCM.pB)[3]){
colnames(DCM.pB[,,z])=DCM.names
	rownames(DCM.pB[,,z])=DCM.names}
}

    PP = PP[-j]
    j = 1:(DCM.n*DCM.m)
    DCM.pC = t(matrix(PP[j],DCM.n,DCM.m))
if(length(DCM.names)==DCM.n) {
    colnames(DCM.pC)=DCM.names
    rownames(DCM.pC) = names(DCM.ons)}
    DCM.F = F
    DCM.Ce = as.matrix(S)
    DCM
   
    }

In [ ]:
def HPF(DCM){
    if(DCM.HPF > 0):
        n = floor(2*(DCM.v*DCM.TR)/DCM.HPF + 1)
        K = spm_dctmtx(DCM.v, n)[,-1]
        return K
    else:
        return 1

In [ ]:
def Ncdf(u,v):
    x=0
    u=as.matrix(abs(u))
    v=as.matrix(v)
    ad=as.matrix(c(2,2,2))
    rd=max(ad)
    as1=c(1,1,rep(1,(rd-ad[1])))
    as2=c(dim(u),rep(1,(rd-ad[2])))
    as3=c(dim(v),rep(1,(rd-ad[3])))
    as=rbind(as1,as2,as3)
    rs=pmax(as1,as2,as3)
    xa=rowMeans(as)>1
    F=rep(0,max(rs))
    md=v>0
    Q=which(v>0)
    if (xa[1]):
        Qx=Q
    else:
        Qx=1
    if (xa[2]):
        Qu=Q
    else:
        Qu=1
    if (xa[3]):
        Qv=Q
    else:
        Qv=1

    X=(x[Qx]-u[Qu])/sqrt(2*v[Qv])
    F[Q]=1-(0.5 + 0.5*(2*pnorm(X*sqrt(2))-1))
    return F


In [ ]:
def Glover(DCM,ts):
    Kernel = spm_kernels(DCM)
#     fill with 0 v rows, n cols
    na = matrix(0,DCM.v,DCM.n)   
    for i in range(1,DCM.n + 1):
    H = fft(Kernel[,i,1])
    M = fft(ts[,i])
    N0 = DCM.v*DCM.Ce[(i-1)*DCM.v+1,(i-1)*DCM.v+1]
    na[,i] = as.real(as.real(fft(Conj(H)*M)/(as.real(H*Conj(H))+N0),inverse=TRUE))/DCM.v
    #na=as.matrix(na)
    return na


In [ ]:
def spm_svd(X,tol):
    X = as.matrix(X)
    M = dim(X)[1]
    N = dim(X)[2]
    p = which(rowSums(X)!=0)
    q = which(colSums(X)!=0)
    X = X[p,q]
    i = which(X!=0,arr.ind=TRUE)[,1]
    j = which(X!=0,arr.ind=TRUE)[,2]
    
    s = X[X!=0]
    M_ = dim(X)[1]
    N_ = dim(X)[2]
    ve = svd(X).v
    uu = svd(X).u
    S = diag(svd(X).d)
    s = diag(S)**2
    j = which(s*length(s)/sum(s) >= tol & s >= 0)
    ve = ve[,j]
    uu = uu[,j]
    S = S[j,j]
    j = length(j)
    U = matrix(0,M,j)
    V = matrix(0,N,j)
    
    V[q,] = ve
    return V
    
    
def spm_int(PM,DCM):
    x_exp = rbind(1,as.matrix(rep(0,DCM.x)))
    # PM=c(0,DCM.a,DCM.b,DCM.c,DCM.H%x%rep(1,DCM.n))
       
    # Integrate
    bireduce=spm_bireduce(PM,DCM.x,DCM.n,DCM.m,DCM.TE)
    M0 = bireduce.M0
    M1 = bireduce.M1

    # Generate Bold signal
    y = matrix(0,DCM.n,DCM.v)
    dy = y
  
    for i in len(DCM.T0):
        u = DCM.sf[DCM.T0[i],:]
        if (DCM.s[i]>DCM.v):
            J = M0
            for (j in 1:DCM.m){
            J = J+u[j]*M1[,,j]
        else:
            y[,DCM.s[i]] = funl2(x_exp[2:length(x_exp)], PM,DCM.n,DCM.TE)
                
        x_exp = spm_expm2((J*DCM.dt0[i]),x_exp)          
     
    return t(y)
 